In [14]:
import Simulation.pys.para.model_para as Mp
import Simulation.pys.full_sim.Catalogue_one_shell as cata_shell
from time import time
from importlib import reload
import pickle as pkl

In [15]:
with open('../../data/test_model/test_cosmo_pack.pkl', 'rb') as f:
    cosmo_pack = pkl.load(f)

with open('../../data/test_model/test_dust_pack.pkl', 'rb') as f:
    dust_pack = pkl.load(f)

In [16]:
cosmo_pack.keys()

dict_keys(['density_map', 'cl_label', 'cl_shell', 'cosmo_p'])

In [18]:
obs_f = {"EBV_in": dust_pack['EBV_in'], #np.zeros_like(EBV_in),
         "EBV_out": Mp.SFD, #np.zeros_like(EBV_out),
         "Band_coefficient_in": dust_pack['band_coefficient_Rv'],
         "Band_coefficient_out": dust_pack['band_coefficient_ccm_31'],
         "ErrorModel_clean": Mp.errModel_clean,
         "ErrorModel_obs":Mp.errModel_dust,
         "nside":Mp.nside,
         "photoz_est": [Mp.photo_z_estimator_RF],
         "Rv_map":  dust_pack["Rv_map"],
         "mask": Mp.SFD<0.2,
         "zbin_in": Mp.zbin_in,
         "zbin_out": Mp.zbin_out}

In [19]:
#From density map to Ncount expectation map
Ncount_Exp_shell_map = cata_shell.ExpNcount_map_shells(cosmo_pack["density_map"], Mp.Ncount_hist)

In [20]:
# Operation on one shell
reload(cata_shell)
shell_id = 30
cata_one_shell = cata_shell.Catalogue_one_shell(Ncount_Exp_shell_map[shell_id], Mp.template_cata_shell[shell_id], obs_f)
cata_one_shell.Sample_photometry()

In [21]:
reload(cata_shell)
print("number of galalxy in this shell:", len(cata_one_shell.sampled_catalogue_shell))
cata_one_shell.Add_extinction_info()
cata_one_shell.Add_Exintction()
time1 = time()
cata_one_shell.Add_noise(dust_noise = True, clean_noise = True)
time2 = time()
print("Time usage for adding noise: ", time2-time1)
cata_one_shell.Dereddening()
cata_one_shell.Estimate_photo_z(choice = "dust")
time3 = time()
print("Time usage for estimating photo z: ", time3-time2)
cata_one_shell.Estimate_photo_z(choice = "clean")


number of galalxy in this shell: 3765996
Time usage for adding noise:  6.388761758804321
Time usage for estimating photo z:  8.748862028121948


In [22]:
hist_dust = cata_one_shell.Statistics_on_shell(Mp.zbin_out, "dust")
hist_clean = cata_one_shell.Statistics_on_shell(Mp.zbin_out, "clean")

In [23]:
results = cata_shell.Simulate_shells(cosmo_pack["density_map"],Mp.Ncount_hist,Mp.template_cata_shell,
                           obs_f, Mp.zbin_out,choice = "both", expectation_total_sample=2e8)

100%|██████████| 40/40 [22:45<00:00, 34.14s/it]


In [24]:
# with open('../../data/test_model/test_full_sim_result_pack1.pkl', 'wb') as f:
#     pack = {"cosmo_pack": cosmo_pack, "dust_pack": dust_pack, "data": results, "mask":obs_f["mask"]}
#     pkl.dump(pack, f)